In [393]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from datetime import datetime 
from datetime import date as date
from calendar import monthrange

In [394]:
meter = pd.read_excel("BulkMeterData.xlsx")

In [395]:
meter.head()

,DateTime,Power (kW)
0,2018-10-14 00:30:00,10.1980
1,2018-10-14 01:00:00,8.2462
2,2018-10-14 01:30:00,10.1980
3,2018-10-14 02:00:00,12.0000
4,2018-10-14 02:30:00,8.9443


In [396]:
meter_by_hour = meter.groupby(np.arange(len(meter))//2).agg({'DateTime':['min'],'Power (kW)':['sum']})
meter_by_hour.columns = meter_by_hour.columns.get_level_values(0)
meter_by_hour.head()

,DateTime,Power (kW)
0,2018-10-14 00:30:00,18.4442
1,2018-10-14 01:30:00,22.1980
2,2018-10-14 02:30:00,18.9443
3,2018-10-14 03:30:00,24.7582
4,2018-10-14 04:30:00,20.3960


In [397]:
meter_by_hour['Date'] = meter_by_hour['DateTime']
meter_by_hour['Hour'] = meter_by_hour['Date'].apply(lambda x: x.hour)
meter_by_hour['Date'] = meter_by_hour['Date'].apply(lambda x: x.date())

In [398]:
meter_by_hour['Year'] = meter_by_hour['Date'].apply(lambda x: x.year)
meter_by_hour['Month'] = meter_by_hour['Date'].apply(lambda x: x.month)
meter_by_hour['Day'] = meter_by_hour['Date'].apply(lambda x: x.day)
meter_by_hour.head()

,DateTime,Power (kW),Date,Hour,Year,Month,Day
0,2018-10-14 00:30:00,18.4442,2018-10-14,0,2018,10,14
1,2018-10-14 01:30:00,22.1980,2018-10-14,1,2018,10,14
2,2018-10-14 02:30:00,18.9443,2018-10-14,2,2018,10,14
3,2018-10-14 03:30:00,24.7582,2018-10-14,3,2018,10,14
4,2018-10-14 04:30:00,20.3960,2018-10-14,4,2018,10,14


In [399]:
weather = pd.read_excel("CapeTownWeather.xlsx")
weather.head()

,Year,Month,Day,Hour,Minute,Temperature,Relative Humidity,Total Precipitation,Wind Speed,Wind Direction
0,2018,10,14,0,0,17.82,65,0.0,8.89,148.24
1,2018,10,14,1,0,17.86,66,0.0,6.61,150.64
2,2018,10,14,2,0,17.96,66,0.0,3.67,168.69
3,2018,10,14,3,0,17.72,68,0.0,9.75,4.24
4,2018,10,14,4,0,17.01,71,0.0,20.36,351.87


In [400]:
df = pd.merge(meter_by_hour,weather,how='right',left_on=['Year','Month','Day','Hour'], right_on = ['Year','Month','Day','Hour'])

In [401]:
df[8736:]['DateTime'] = datetime(2019,10,13)
df[8736:]['Date'] = df[8736:]['DateTime'].apply(lambda x:x.date())
df.drop(['DateTime','Month','Year','Day','Hour','Minute'],axis=1,inplace=True)

In [402]:
df = df.groupby('Date').agg({'Power (kW)':['sum'],'Temperature':['min','mean','max'],'Relative Humidity':['min','mean','max'],'Total Precipitation':['sum'],'Wind Speed':['min','mean','max'],'Wind Direction':['min','mean','max']})
df.columns = ['Power (kW)','Temperature_min','Temperature_mean','Temperature_max','Relative Humidity_min','Relative Humidity_mean','Relative Humidity_max','Total Precipitation','Wind Speed_min','Wind Speed_mean','Wind Speed_max','Wind Direction_min','Wind Direction_mean','Wind Direction_max']
df.reset_index(inplace=True)
df.head()

,Date,Power (kW),Temperature_min,Temperature_mean,Temperature_max,Relative Humidity_min,Relative Humidity_mean,Relative Humidity_max,Total Precipitation,Wind Speed_min,Wind Speed_mean,Wind Speed_max,Wind Direction_min,Wind Direction_mean,Wind Direction_max
0,2018-10-14,898.8331,15.01,16.321667,17.96,65,79.500000,90,0.5,3.67,28.852500,40.09,4.24,239.685000,357.77
1,2018-10-15,763.6253,14.65,16.349583,18.96,61,75.041667,90,0.0,11.12,15.944583,23.69,151.11,253.496667,313.60
2,2018-10-16,767.1694,14.15,17.294583,21.39,47,66.666667,84,0.0,12.57,21.788750,30.91,139.40,157.974583,176.48
3,2018-10-17,736.7393,13.69,18.419583,23.16,38,57.666667,80,0.0,8.53,20.685000,32.11,150.10,156.108750,162.51
4,2018-10-18,757.6639,16.37,19.243333,23.42,40,62.708333,85,0.0,1.08,9.467917,16.80,108.43,216.287500,360.00


In [403]:
#df['Days_in_month'] = df['Date'].apply(lambda x: monthrange(x.year,x.month)[1])
df['holiday'] = df['Date'].apply(lambda x: 1 if date(x.year,x.month,x.day) in holidays.ZA() else 0)
df['num_of_week'] = df['Date'].apply(lambda x: x.isocalendar()[1])
df['which_weekday'] = df['Date'].apply(lambda x: x.isocalendar()[2])
df['Month'] = df['Date'].apply(lambda x: x.month)
df.head()

,Date,Power (kW),Temperature_min,Temperature_mean,Temperature_max,Relative Humidity_min,Relative Humidity_mean,Relative Humidity_max,Total Precipitation,Wind Speed_min,Wind Speed_mean,Wind Speed_max,Wind Direction_min,Wind Direction_mean,Wind Direction_max,holiday,num_of_week,which_weekday,Month
0,2018-10-14,898.8331,15.01,16.321667,17.96,65,79.500000,90,0.5,3.67,28.852500,40.09,4.24,239.685000,357.77,0,41,7,10
1,2018-10-15,763.6253,14.65,16.349583,18.96,61,75.041667,90,0.0,11.12,15.944583,23.69,151.11,253.496667,313.60,0,42,1,10
2,2018-10-16,767.1694,14.15,17.294583,21.39,47,66.666667,84,0.0,12.57,21.788750,30.91,139.40,157.974583,176.48,0,42,2,10
3,2018-10-17,736.7393,13.69,18.419583,23.16,38,57.666667,80,0.0,8.53,20.685000,32.11,150.10,156.108750,162.51,0,42,3,10
4,2018-10-18,757.6639,16.37,19.243333,23.42,40,62.708333,85,0.0,1.08,9.467917,16.80,108.43,216.287500,360.00,0,42,4,10


In [405]:
X = df.drop(['Power (kW)','Date'],axis=1)
y = df['Power (kW)']

In [406]:
X.columns

Index(['Temperature_min', 'Temperature_mean', 'Temperature_max',
       'Relative Humidity_min', 'Relative Humidity_mean',
       'Relative Humidity_max', 'Total Precipitation', 'Wind Speed_min',
       'Wind Speed_mean', 'Wind Speed_max', 'Wind Direction_min',
       'Wind Direction_mean', 'Wind Direction_max', 'holiday', 'num_of_week',
       'which_weekday', 'Month'],
      dtype='object')

In [407]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
#from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsRegressor
import xgboost
from sklearn.model_selection import ParameterGrid
import warnings
warnings.filterwarnings("ignore")

In [428]:
for_standard = ['Temperature_min', 'Temperature_mean', 'Temperature_max',
       'Relative Humidity_min', 'Relative Humidity_mean',
       'Relative Humidity_max', 'Total Precipitation', 'Wind Speed_min',
       'Wind Speed_mean', 'Wind Speed_max', 'Wind Direction_min',
       'Wind Direction_mean', 'Wind Direction_max']
#for_minmax = ['num_of_week']
for_onehot = ['which_weekday','Month','num_of_week']

In [436]:
def ML_pipeline_rf_GridSearchCV(X,y,random_state,n_folds):
    '''
    Random forest regressor to predict power usage
    '''
    X_other, X_test, y_other, y_test = train_test_split(X, y, test_size=0.20, random_state = random_state)
    kf = KFold(n_splits=n_folds,shuffle=True,random_state=random_state)
    #kf = StratifiedKFold(n_splits=n_folds,shuffle=True,random_state=random_state)
    standard_transformer = Pipeline(steps=[('standard', StandardScaler())])
    minmax_transformer = Pipeline(steps=[('minmax', MinMaxScaler())])
    onehot_transformer = Pipeline(steps=[('onehot', OneHotEncoder(sparse=False,handle_unknown='ignore'))])
    preprocessor = ColumnTransformer(
    transformers=[
        ('standard', standard_transformer, for_standard),
        #('minmax',minmax_transformer,for_minmax),
        ('onehot',onehot_transformer,for_onehot)])
    rf = Pipeline(steps=[('preprocessor', preprocessor),
                          ('Regressor', RandomForestRegressor())])
    param_grid = { 
    'Regressor__max_features': ['auto'],
    'Regressor__max_depth' : range(15,100,10),
    'Regressor__min_samples_split' : range(2,12,2),
    'Regressor__n_estimators' : [200,300],
    'Regressor__random_state' : [random_state]}
    
    grid = GridSearchCV(rf, param_grid=param_grid,scoring = 'neg_mean_squared_error',
                        cv=kf, return_train_score = True,iid=True)
    grid.fit(X_other, y_other)
    
    return grid, grid.score(X_test, y_test)

In [437]:
test_scores_rf = []
grids = []
for i in range(10):
    grid, test_score = ML_pipeline_rf_GridSearchCV(X[:327],y[:327],(i+1)*42,5)
    print(grid.best_params_)
    print('test score:',test_score)
    grids.append(grid)
    test_scores_rf.append(test_score)
print('test MSE:',np.around(np.mean(test_scores_rf),3),'+/-',np.around(np.std(test_scores_rf),3))

{'Regressor__max_depth': 15, 'Regressor__max_features': 'auto', 'Regressor__min_samples_split': 10, 'Regressor__n_estimators': 300, 'Regressor__random_state': 42}
test score: -2552.131100836523
{'Regressor__max_depth': 25, 'Regressor__max_features': 'auto', 'Regressor__min_samples_split': 10, 'Regressor__n_estimators': 300, 'Regressor__random_state': 84}
test score: -2671.018796241128
{'Regressor__max_depth': 15, 'Regressor__max_features': 'auto', 'Regressor__min_samples_split': 10, 'Regressor__n_estimators': 200, 'Regressor__random_state': 126}
test score: -3186.1200343648766
{'Regressor__max_depth': 25, 'Regressor__max_features': 'auto', 'Regressor__min_samples_split': 8, 'Regressor__n_estimators': 300, 'Regressor__random_state': 168}
test score: -2798.938713563171
{'Regressor__max_depth': 15, 'Regressor__max_features': 'auto', 'Regressor__min_samples_split': 10, 'Regressor__n_estimators': 300, 'Regressor__random_state': 210}
test score: -1936.9591481627872
{'Regressor__max_depth': 2

In [438]:
sum((grids[7].predict(X[327:364]) - y[327:364])**2)/37

6736.600156308068

In [439]:
grids[7].predict(X[364:])

array([744.70753208])

In [450]:
def ML_pipeline_xgb_GridSearchCV(X,y,random_state,n_folds):
    '''
    XGB regressor to predict power usage
    '''
    X_other, X_test, y_other, y_test = train_test_split(X, y, test_size=0.15, random_state = random_state)
    kf = KFold(n_splits=n_folds,shuffle=True,random_state=random_state)
    standard_transformer = Pipeline(steps=[('standard', StandardScaler())])
    #minmax_transformer = Pipeline(steps=[('minmax', MinMaxScaler())])
    onehot_transformer = Pipeline(steps=[('onehot', OneHotEncoder(sparse=False,handle_unknown='ignore'))])
    XGB = xgboost.XGBRegressor()
    preprocessor = ColumnTransformer(
    transformers=[
        ('standard', standard_transformer, for_standard),
        #('minmax',minmax_transformer,for_minmax),
        ('onehot',onehot_transformer,for_onehot)])
    reg = Pipeline(steps=[('preprocessor', preprocessor),
                          ('Regressor', XGB)])
    param_grid = { 
    'Regressor__learning_rate': [0.023],
    'Regressor__n_estimators': [300,400],
    'Regressor__max_depth': [3,6,10],
    'Regressor__colsample_bytree': [0.8],  
    'Regressor__random_state' : [random_state]}
    
    grid = GridSearchCV(reg, param_grid=param_grid,scoring = 'neg_mean_squared_error',
                        cv=kf, return_train_score = True,iid=True)
    grid.fit(X_other, y_other)
    
    return grid, grid.score(X_test, y_test)

In [451]:
import warnings
warnings.filterwarnings("ignore")
test_scores_xgb = []
grids = []
for i in range(10):
    grid, test_score = ML_pipeline_xgb_GridSearchCV(X[:327],y[:327],(i+1)*42,5)
    print(grid.best_params_)
    print('test score:',test_score)
    grids.append(grid)
    test_scores_xgb.append(test_score)
print('test MSE:',np.around(np.mean(test_scores_xgb),3),'+/-',np.around(np.std(test_scores_xgb),3))

[17:04:36] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:04:36] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:04:37] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:04:37] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:04:37] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:04:37] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:04:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:04:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:04:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

In [464]:
sum((grids[6].predict(X[327:364]) - y[327:364])**2)/37

7312.632341258347

In [465]:
grids[6].predict(X[364:])

array([781.83014], dtype=float32)